<a href="https://colab.research.google.com/github/Glasiermedic/Natural-Language-process/blob/master/wes_bible_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import urllib.request

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
#from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from IPython.display import display

import spacy
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
nltk.download('gutenberg')

from spacy.lang.en import English

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from sklearn.metrics import confusion_matrix
#from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.neighbors import KNeighborsRegressor

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import LabelEncoder, Imputer

from timeit import default_timer as timer

import os

import pydotplus
from sklearn import tree
from sklearn import preprocessing
%matplotlib inline
sns.set_style('white')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [0]:
from gensim import corpora, similarities
from gensim.models import LsiModel

#Natural Language Author Processing
## Identifying the voice of words

#### After importing and prepping the document(s) for processing/cleaning we will utilize a few variations on our model using first a Bag of Words approach and then a latent semantic indexing model. 

#### To get the document for processing you first have to download it from a trusted source (nltk, gutenberg is a known quantity ))   

#### We also will need the 'stopwords' to help clean our data and make processing much easier in the long run

In [0]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
nlp_spacy = spacy.load('en')

###Cleaning our text
#### We will remove all titles, chapter headings, white space, 

In [0]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = clean_punct(text)
    text = ' '.join(text.split())
    
    return text

In [0]:
def clean_punct(text):
    text = text.replace('"','')
    text = text.replace('“','')
    text = text.replace('”','')
    text = text.replace('\'','')
    text = text.replace('.',' .')
    text = text.replace(',',' ')
    text = text.replace('?',' ?')
    text = text.replace('!',' !')
    text = text.replace(':',' :')
    text = text.replace(';',' ;')
    text = text.replace('(','( ')
    text = text.replace(')',' )')
    text = text.replace('-', ' - ')
    return text

def get_texts(text_url):
    values = {'q' : 'python programming tutorials'}
    data = urllib.parse.urlencode(values)
    data = data.encode('utf-8') # data should be bytes
    req = urllib.request.Request(text_url, data)
    resp = urllib.request.urlopen(req)
    respData = resp.read()
    text = respData.decode("utf-8")
    text = text.replace('_','')
    text = text.replace('\xa0', '')
    text = text.replace(',—', ', ')
    text = text.replace('--', ' - ')
    text = clean_punct(text)
    return text
    
def get_paragraphs(text_url):
    values = {'q' : 'python programming tutorials'}
    data = urllib.parse.urlencode(values)
    data = data.encode('utf-8') # data should be bytes
    req = urllib.request.Request(text_url, data)
    resp = urllib.request.urlopen(req)
    respData = resp.read()
    text = respData.decode("utf-8")
    text = text.replace('_','')
    text = text.replace('\xa0', '')
    text = text.replace(',—', ', ')
    text = text.replace('--', ' - ')
    text = clean_punct(text)
    paragraphs = BlanklineTokenizer().tokenize(text)
    cleaned_paras = []
    for para in paragraphs:
        cleaned_paras.append(para.replace("\r\n", " "))
    return cleaned_paras

In [0]:
def lemma_frequencies(text, include_stop=False):
  lemmas = []
  for token in text:
    if not token.is_punct and (not token.is_stop or include_stop):
      lemmas.append(token.lemma_)
            
    # Build and return a Counter object containing word counts.
  return Counter(lemmas)


#### importing the text files from the gutenberg corpus from local and internet sources

#### prior to running spaCy on the texts we need to verify that they are not too big (greater than 300000 characters after cleaning) so that our kernal doesn't crash

In [0]:
aver_list = 0

### William Shakespeare texts

In [0]:

caesar  = gutenberg.raw('shakespeare-caesar.txt')[-61118:]
caesar = text_cleaner(caesar)
print(caesar[0:100])
print ("length of caesar: {}".format(len(caesar)))
aver_list += len(caesar)
hamlet  = gutenberg.raw('shakespeare-hamlet.txt')[-61118:]
hamlet = text_cleaner(hamlet)
print ("length of hamlet: {}".format(len(hamlet)))
aver_list += len(hamlet)
macbeth = gutenberg.raw('shakespeare-macbeth.txt')[-61118:]
macbeth = text_cleaner(macbeth)
print ("length of macbeth: {}".format(len(macbeth)))
aver_list += len(macbeth)

What is now amisse That Caesar and his Senate must redresse ? Metel . Most high most mighty and most
length of caesar: 59303
length of hamlet: 59549
length of macbeth: 59451


### Jane Austin texts

In [0]:
emma = gutenberg.raw('austen-emma.txt')[-61118:]
emma = text_cleaner(emma)
print ("length of emma: {}".format(len(emma)))
aver_list += len(emma)
persuasion = gutenberg.raw('austen-persuasion.txt')[-61118:]
persuasion = text_cleaner(persuasion)
print ("length of persuasion: {}".format(len(persuasion)))
aver_list += len(persuasion)
sense = gutenberg.raw('austen-sense.txt')[-61118:]
sense = text_cleaner(sense)
print ("length of sense: {}".format(len(sense)))
aver_list += len(sense)

length of emma: 60041
length of persuasion: 60210
length of sense: 59930


### H.G. Wells texts

In [0]:
time_machine = get_texts('http://www.gutenberg.org/files/35/35-0.txt')[-61118:]
print ("length of time_machine: {}".format(len(time_machine)))
aver_list += len(time_machine)
war_o_worlds = get_texts('http://www.gutenberg.org/files/36/36-0.txt')[-61118:]
print ("length of war_o_worlds: {}".format(len(war_o_worlds)))
aver_list += len(war_o_worlds)
island_moreau = get_texts('http://www.gutenberg.org/cache/epub/159/pg159.txt')[-61118:]
print ("length of island_moreau: {}".format(len(island_moreau)))
aver_list += len(island_moreau)

aver =aver_list/9

length of time_machine: 61118
length of war_o_worlds: 61118
length of island_moreau: 61118


In [0]:
aver_list += len(emma)
aver_list += len(persuasion)
aver_list += len(sense)
aver_list += len(caesar)
aver_list += len(hamlet)
aver_list += len(macbeth)
aver_list += len(time_machine)
aver_list += len(war_o_worlds)
aver_list += len(island_moreau)
print((aver_list/9)*.02)
aver = (aver_list/9)*.02

2408.168888888889


### Dickens texts

### Lewis Carroll texts

### Arthur Conan Doyle texts

### List of books used

In [0]:
%%time
caesar_doc = nlp_spacy(caesar)

CPU times: user 1.51 s, sys: 156 ms, total: 1.67 s
Wall time: 1.69 s


In [0]:
%%time
hamlet_doc = nlp_spacy(hamlet)

CPU times: user 1.58 s, sys: 34.4 ms, total: 1.61 s
Wall time: 1.61 s


In [0]:
%%time
macbeth_doc = nlp_spacy(macbeth)

CPU times: user 1.53 s, sys: 29.4 ms, total: 1.56 s
Wall time: 1.56 s


In [0]:
%%time
emma_doc = nlp_spacy(emma)

CPU times: user 1.49 s, sys: 39.9 ms, total: 1.53 s
Wall time: 1.54 s


In [0]:
%%time
persuasion_doc = nlp_spacy(persuasion)

CPU times: user 1.45 s, sys: 35 ms, total: 1.49 s
Wall time: 1.49 s


In [0]:
%%time
sense_doc = nlp_spacy(sense)

CPU times: user 1.4 s, sys: 30.2 ms, total: 1.43 s
Wall time: 1.43 s


In [0]:
%%time
time_doc = nlp_spacy(time_machine)

CPU times: user 1.6 s, sys: 42.4 ms, total: 1.64 s
Wall time: 1.65 s


In [0]:
%%time
war_doc = nlp_spacy(war_o_worlds)

CPU times: user 1.56 s, sys: 39.4 ms, total: 1.6 s
Wall time: 1.61 s


In [0]:
%%time
island_doc = nlp_spacy(island_moreau)

CPU times: user 1.59 s, sys: 44.6 ms, total: 1.64 s
Wall time: 1.65 s


In [0]:
import random
random.seed(1)


caesar_sents = [[sent, "Shakespeare", "Caesar"] for sent in caesar_doc.sents]
hamlet_sents = [[sent, "Shakespeare", "Hamlet"] for sent in hamlet_doc.sents]
macbeth_sents = [[sent, "Shakespeare", "Macbeth"] for sent in macbeth_doc.sents]
emma_sents = [[sent, "Austin", "Emma"] for sent in emma_doc.sents]
persuasion_sents = [[sent, "Austin", "Persuasion"] for sent in persuasion_doc.sents]
sense_sents = [[sent, "Austin", "Sense and Sensability"] for sent in sense_doc.sents]
time_sents = [[sent, "Wells", "Time Machine"] for sent in time_doc.sents]
war_sents = [[sent, "Wells", "War of the Worlds"] for sent in war_doc.sents]
island_sents = [[sent, "Wells", "Island of Moreau"] for sent in island_doc.sents]


In [0]:
print (len(caesar_sents))
#caesar_sents = random.sample(caesar_sents,(len(caesar_sents)//3))
len(caesar_sents)

1477


1477

In [0]:
sentences = pd.DataFrame(caesar_sents + hamlet_sents + macbeth_sents + emma_sents
                         + persuasion_sents + sense_sents + time_sents +
                         war_sents + island_sents)
sentences.head()

,0,1,2
0,"(What, is, now, amisse, That, Caesar, and, his...",Shakespeare,Caesar
1,"(Metel, .)",Shakespeare,Caesar
2,"(Most, high, most, mighty, and, most, puisant)",Shakespeare,Caesar
3,"(Caesar, Metellus, Cymber, throwes, before, th...",Shakespeare,Caesar
4,"(I, must, preuent, thee, Cymber, :, These, cou...",Shakespeare,Caesar


### Using our nine texts we now have 9816 sentences

In [0]:
sentences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8732 entries, 0 to 8731
Data columns (total 3 columns):
0    8732 non-null object
1    8732 non-null object
2    8732 non-null object
dtypes: object(3)
memory usage: 204.7+ KB


### Bag of Words functions

In [0]:
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(300)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words, title):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 5 == 0:
            print("Processing row {}".format(i))
            
    return df

In [0]:
doc_list = [caesar_doc, hamlet_doc, macbeth_doc, emma_doc, persuasion_doc, sense_doc, time_doc, war_doc, island_doc ]

In [0]:
common_words = set()
for doc in doc_list:
  bow1 = bag_of_words(doc)
  common_words.update(bow1)
  print(len(common_words))
  

300
476
603
824
940
1046
1233
1305
1369


In [0]:
%%time
word_counts = bow_features(sentences, common_words, sentences[2])
#word_counts = pd.read_csv('https://github.com/Glasiermedic/Natural-Language-process/blob/master/data.csv')
word_counts.head(100)

Processing row 0
Processing row 5
Processing row 10
Processing row 15
Processing row 20
Processing row 25
Processing row 30
Processing row 35
Processing row 40
Processing row 45
Processing row 50
Processing row 55
Processing row 60
Processing row 65
Processing row 70
Processing row 75
Processing row 80
Processing row 85
Processing row 90
Processing row 95
Processing row 100
Processing row 105
Processing row 110
Processing row 115
Processing row 120
Processing row 125
Processing row 130
Processing row 135
Processing row 140
Processing row 145
Processing row 150
Processing row 155
Processing row 160
Processing row 165
Processing row 170
Processing row 175
Processing row 180
Processing row 185
Processing row 190
Processing row 195
Processing row 200
Processing row 205
Processing row 210
Processing row 215
Processing row 220
Processing row 225
Processing row 230
Processing row 235
Processing row 240
Processing row 245
Processing row 250
Processing row 255
Processing row 260
Processing row 

In [0]:
word_counts.shape

(8732, 1371)

In [0]:
#from google.colab import drive
#drive.mount('drive')



In [0]:
#word_counts.to_csv('bow_data.csv')
#!cp data.csv drive/My\ Drive/

In [0]:
#!ls

In [0]:
print(sentences[0][0])

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case 
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

# Applying the vectorizer
X_tdif = vectorizer.fit_transform(sentences[0].astype(str))
Y_tdif = sentences[1]

X_train, X_test, y_train, y_test= train_test_split(X_tdif, 
                                                    Y_tdif,
                                                    test_size=0.4,
                                                    random_state=0)

# Reshapes the vectorizer output into something people can read
X_train_csr = X_train.tocsr()

#number of paragraphs
n = X_train_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bysent = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()


for i, j in zip(*X_train_csr.nonzero()):
    tfidf_bysent[i][terms[j]] = X_train_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Tf_idf vector:', tfidf_bysent[0])

What is now amisse That Caesar and his Senate must redresse ?
Tf_idf vector: {'durst': array(1.)}


In [0]:
rfc = ensemble.RandomForestClassifier()
Y_bow = word_counts['text_source']
X_bow = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, 
                                                    Y_bow,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train_bow, y_train_bow)

print('Training set score:', rfc.score(X_train_bow, y_train_bow))
print('\nTest set score:', rfc.score(X_test_bow, y_test_bow))
pd.crosstab(y_train_bow, rfc.predict(X_train_bow))

Training set score: 0.9530444741362856

Test set score: 0.8740337818494132


col_0,Austin,Shakespeare,Wells
text_source,,,
Austin,1068,114,8
Shakespeare,23,2548,13
Wells,7,81,1377


In [0]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
kfolds = StratifiedShuffleSplit(n_splits =5, random_state=1)

from sklearn.svm import SVC as svc
from sklearn.linear_model import LogisticRegression as lr
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import GradientBoostingClassifier as xboost
#from sklearn.svm import LinearSVC as svc
from sklearn.pipeline import Pipeline

#xboost = ensemble.GradientBoostingClassifier()
#rfc = ensemble.RandomForestClassifier()
#lr = LogisticRegression()

rfc_pipe = Pipeline([('classifier' , rfc())])
xboost_pipe = Pipeline([('classifier' , xboost())])
lr_pipe = Pipeline([('classifier' , lr())])
svc_pipe = Pipeline([('classifier' , svc())])

#pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [rfc(n_jobs=-1)],
     'classifier__n_estimators' : list(range(10,101,10)),
     'classifier__max_features' : list(range(6,32,5))},
    {'classifier' : [lr(n_jobs=-1)],
     'classifier__penalty' : ['l1', 'l2'],
     'classifier__C' : np.logspace(3, 4, 5),
     'classifier__solver' : ['liblinear']},
    {'classifier' : [svc()],
     'classifier__C' : [0.001, 0.01, 1, 10],
     'classifier__gamma' : [0.001, 0.01, 1]}, 
    {'classifier' : [xboost()],
     'classifier__subsample' : [0.6, 0.8, 1.0],
     'classifier__max_depth' : [3, 4, 5] }
]






# Create grid search object

clf = GridSearchCV(rfc_pipe, param_grid = param_grid, cv = kfolds, verbose=True, n_jobs=-1 )

# Fit on data

#best_clf = clf.fit(X_train_bow, y_train_bow)
#best_clf.best_score_
#best_clf.best_estimator_

In [0]:
#best_clf.best_estimator_
from sklearn.metrics import classification_report


#### Using a pipeline we can tune the variety of models to get the best results for our Author detector model. 

In [63]:
#X_bow  = "Wes"
#Y_bow  = .9 
#X_tdif = "Dusty"
#Y_tdif = .5
#rfc_pipe = "1"
#lr_pipe = "2"

preprocess_titles = ['BOW', "TDIDF"]
prepprocess_data_X_Y = [(X_bow, Y_bow), (X_tdif, Y_tdif)]
#ml_model_titles = ['rfc', 'tuc', 'utg']
ml_model_list = [rfc_pipe]
#test_df =pd.DataFrame()


def model_pipeline(preprocess_titles, preprocess_data_X_Y, ml_model_titles, ml_models):
  mod_res =[]
  for pre in preprocess_titles: 
    print ("  --- {} ---  ".format(pre))
    X,Y = preprocess_data_X_Y[preprocess_titles.index(pre)]
    #print ("{} model: {}".format(model, Y))
    X_train, X_test, y_train, y_test = train_test_split(X,Y,
                                                    test_size=0.4,
                                                    random_state=1)
    best_clf = clf.fit(X_train, y_train)
    best_scores = best_clf.best_score_
    best_estimate = best_clf.best_estimator_
    mod_res.append((pre, best_scores, best_estimate))
      
    print (mod_res)
    results_df = pd.DataFrame(mod_res, columns = ['preprocess', 'results', 'estimator'])
    #results_df.append({'preprocess': pre, model: ml_models[ml_model_titles.index(model)], best.fit.}, ignore_index=True)
    print ("  --- {} ---  ".format(pre))
    print(pd.crosstab(best_clf.predict(X_test),y_test))
    print("")
    print(classification_report(best_clf.predict(X_test), y_test))
            
    
  return results_df
    

test_df =model_pipeline(['BOW', "TDIDF"], [(X_bow, Y_bow), (X_tdif, Y_tdif)], ['random_forest', 'logistic_regression'], [rfc_pipe, lr_pipe])

  --- BOW ---  
Fitting 5 folds for each of 91 candidates, totalling 455 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 94.4min
[Parallel(n_jobs=-1)]: Done 455 out of 455 | elapsed: 110.2min finished


[('BOW', 0.8923664122137405, Pipeline(memory=None,
         steps=[('classifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=6, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=80, n_jobs=-1,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False))]
  --- BOW ---  
text_source  Austin  Shakespeare  Wells
row_0                                  
Austin          601           60     16
Shakespeare     188         163

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 18.3min
[Parallel(n_jobs=-1)]: Done 455 out of 455 | elapsed: 19.1min finished


[('BOW', 0.8923664122137405, Pipeline(memory=None,
         steps=[('classifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=6, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=80, n_jobs=-1,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)), ('TDIDF', 0.8393129770992367, Pipeline(memory=None,
         steps=[('classifier',
                 SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
           

#### This is an example of the optimal model for author detection. As we add in more words to the "most common words" list we get even more accurate results. 

In [0]:
lr = LogisticRegression()
train = lr.fit(X_train_bow, y_train_bow)
print(X_train_bow.shape, y_train_bow.shape)
print('Training set score:', lr.score(X_train_bow, y_train_bow))
print('\nTest set score:', lr.score(X_test_bow, y_test_bow))
print('cross val:', cross_val_score(lr, X_train_bow, y_train_bow, cv=5))
pd.crosstab(y_train_bow, lr.predict(X_train_bow))

(5239, 1369) (5239,)
Training set score: 0.9205955334987593

Test set score: 0.8972230174634984
cross val: [0.88645038 0.8740458  0.8759542  0.8730916  0.89207259]


col_0,Austin,Shakespeare,Wells
text_source,,,
Austin,961,228,1
Shakespeare,10,2564,10
Wells,19,148,1298


In [0]:
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('cross val:', cross_val_score(lr, X_train, y_train, cv=5))
pd.crosstab(y_train, lr.predict(X_train))